In [1]:
import shutil
from pathlib import Path

import torch
import numpy as np
import matplotlib.pyplot as plt

from lib import (
    load_exp_conf,
    load_exp_data,
    P3innDir,
    pi3nn_compute_PI_and_mean,
    FinnDir,
)

# Run

In [2]:
data_out_root = Path("../data_out").resolve()
assert data_out_root.exists()
pi3nn_root = data_out_root / "p3inn"
finn_root = data_out_root / "finn"
pi3nn_root.mkdir(exist_ok=True, parents=True)
finn_root.mkdir(exist_ok=True, parents=True)

In [3]:
data_core2_df = load_exp_data(name="Core 2")
display(data_core2_df.head())

,time,c_diss
0,0.000000,0.000000
1,0.838329,0.000096
2,1.377912,0.000062
3,1.879996,0.000061
4,2.379996,0.000052


### PI3NN on experimental data to obtain PIs for different quantiles

In [4]:
# p3inn_core2_dir = P3innDir(pi3nn_root / "core2")
# shutil.rmtree(p3inn_core2_dir.path.absolute())
p3inn_core2_dir = P3innDir(pi3nn_root / "core2")
np.save(p3inn_core2_dir.x_data_path, data_core2_df["time"].to_numpy().reshape(-1, 1))
np.save(p3inn_core2_dir.y_data_path, data_core2_df["c_diss"].to_numpy().reshape(-1, 1))

quantiles = [1.0 - 0.05 * i for i in range(1, 10)]
pi3nn_compute_PI_and_mean(
    out_dir=p3inn_core2_dir.path,
    quantiles=quantiles,
)

### FINN on all quantiles to obtain retardations

In [5]:
finn_exp_root = finn_root / "p3inn_core2_new_multiprocessing"
finn_exp_root

PosixPath('/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing')

In [6]:
conf_core2 = load_exp_conf(name="Core 2")
display(conf_core2)

{'D': 2.0045664000000002e-05,
 'por': 0.288,
 'rho_s': 1957.0,
 'X': 0.026034999999999996,
 'T': 39.824440010000004,
 'Nx': 20.0,
 'Nt': 55.0,
 'sample_radius': 0.02375,
 'Q': 0.00010435199999999999,
 'solubility': 1.6,
 'Dirichlet': 0.0,
 'Cauchy': 1.0}

In [7]:
import subprocess
from pathlib import Path

# Prepare a list of commands to run in parallel
commands = []

for quantile, bound_up, bound_down in p3inn_core2_dir.iter_pred_PIs():
    for bound_type, bound in [("up", bound_up), ("down", bound_down)]:
        bound_file = Path(
            f"/tmp/bound_{bound_type}_{quantile:g}.npy"
        )  # Save bound data to file
        np.save(bound_file, bound)


        finn_dir = FinnDir(finn_exp_root / f"{bound_type}_{quantile:g}")
        print(finn_dir.path.resolve())

        command = f"python3 run_finn_core2.py {finn_dir.path.resolve()} {bound_file}"
        commands.append(command)

/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/up_0.55
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.55
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/up_0.6
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.6
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/up_0.65
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.65
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/up_0.7
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.7
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/up_0.75
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.75
/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multip

In [8]:
# Write all commands to a temporary file (input for GNU Parallel)
commands_file = Path("/tmp/parallel_commands.txt")
with open(commands_file, "w") as f:
    f.write("\n".join(commands))
print(commands_file.resolve())

/private/tmp/parallel_commands.txt


In [9]:
command = f"cat {commands_file} | parallel -j 6 --bar"
subprocess.run(command, check=True, shell=True)

sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
0% 0:18=0s pytho

/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.6 is ready to run
Running experiment for /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.6
It   1/21 | mse = 3.02e-08 | loss = 3.29e+01 | dt = 82.9s | lr = 1.0e-01 | loss_mse = 3.0e-03 | loss_phys = 0.0e+00
It   2/21 | mse = 2.49e-08 | loss = 2.92e-03 | dt = 68.3s | lr = 9.7e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   3/21 | mse = 2.48e-08 | loss = 2.48e-03 | dt = 58.5s | lr = 8.9e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   4/21 | mse = 2.48e-08 | loss = 2.48e-03 | dt = 53.2s | lr = 7.7e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   5/21 | mse = 2.46e-08 | loss = 2.48e-03 | dt = 54.2s | lr = 6.3e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   6/21 | mse = 2.49e-08 | loss = 2.46e-03 | dt = 52.2s | lr = 4.7e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   7/21 | mse = 2.42e-08 | loss = 2.49e-03 | dt = 53.3s | lr = 3.3e-02 | loss

5% 1:17=0s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/data_sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=17s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=17s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=24s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=28s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
5% 1:17=35s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/datash: /dev

/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.65 is ready to run
Running experiment for /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.65
It   1/21 | mse = 8.65e-08 | loss = 2.11e+01 | dt = 65.8s | lr = 1.0e-01 | loss_mse = 8.7e-03 | loss_phys = 0.0e+00
It   2/21 | mse = 2.51e-08 | loss = 7.41e-03 | dt = 54.8s | lr = 9.7e-02 | loss_mse = 2.5e-03 | loss_phys = 0.0e+00
It   3/21 | mse = 2.42e-08 | loss = 2.49e-03 | dt = 53.0s | lr = 8.9e-02 | loss_mse = 2.4e-03 | loss_phys = 0.0e+00
It   4/21 | mse = 2.41e-08 | loss = 2.42e-03 | dt = 43.1s | lr = 7.7e-02 | loss_mse = 2.4e-03 | loss_phys = 0.0e+00
It   5/21 | mse = 2.41e-08 | loss = 2.41e-03 | dt = 42.6s | lr = 6.3e-02 | loss_mse = 2.4e-03 | loss_phys = 0.0e+00
It   6/21 | mse = 2.40e-08 | loss = 2.41e-03 | dt = 42.8s | lr = 4.7e-02 | loss_mse = 2.4e-03 | loss_phys = 0.0e+00
It   7/21 | mse = 2.17e-08 | loss = 2.40e-03 | dt = 43.2s | lr = 3.3e-02 | lo

11% 2:16=27m29s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=25m48s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=23m57s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=23m29s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=22m43s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=22m25s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
11% 2:16=22m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev

...

83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
83% 15:3=14m10s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev

/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.9 is ready to run
Running experiment for /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.9
It   1/21 | mse = 3.89e-06 | loss = 4.02e+00 | dt = 562.0s | lr = 1.0e-01 | loss_mse = 3.9e-01 | loss_phys = 3.5e-03
It   2/21 | mse = 4.12e-08 | loss = 3.35e-01 | dt = 384.4s | lr = 9.7e-02 | loss_mse = 4.1e-03 | loss_phys = 0.0e+00
It   3/21 | mse = 3.44e-08 | loss = 3.99e-03 | dt = 189.2s | lr = 8.9e-02 | loss_mse = 3.4e-03 | loss_phys = 0.0e+00
It   4/21 | mse = 3.42e-08 | loss = 3.44e-03 | dt = 141.5s | lr = 7.7e-02 | loss_mse = 3.4e-03 | loss_phys = 0.0e+00
It   5/21 | mse = 3.40e-08 | loss = 3.42e-03 | dt = 154.4s | lr = 6.3e-02 | loss_mse = 3.4e-03 | loss_phys = 0.0e+00
It   6/21 | mse = 3.34e-08 | loss = 3.40e-03 | dt = 138.2s | lr = 4.7e-02 | loss_mse = 3.3e-03 | loss_phys = 0.0e+00
It   7/21 | mse = 3.09e-08 | loss = 3.33e-03 | dt = 87.3s | lr = 3.3e-02 

sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
88% 16:2=10m20s 

/Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.95 is ready to run
Running experiment for /Users/r/Documents/stud/ss24/finnpinn/data_out/finn/p3inn_core2_new_multiprocessing/down_0.95
It   1/21 | mse = 9.33e-08 | loss = 2.92e+01 | dt = 177.2s | lr = 1.0e-01 | loss_mse = 9.3e-03 | loss_phys = 0.0e+00
It   2/21 | mse = 5.23e-08 | loss = 8.52e-03 | dt = 166.2s | lr = 9.7e-02 | loss_mse = 5.2e-03 | loss_phys = 0.0e+00
It   3/21 | mse = 5.16e-08 | loss = 5.22e-03 | dt = 121.1s | lr = 8.9e-02 | loss_mse = 5.2e-03 | loss_phys = 0.0e+00
It   4/21 | mse = 5.16e-08 | loss = 5.16e-03 | dt = 75.8s | lr = 7.7e-02 | loss_mse = 5.2e-03 | loss_phys = 0.0e+00
It   5/21 | mse = 5.14e-08 | loss = 5.16e-03 | dt = 74.0s | lr = 6.3e-02 | loss_mse = 5.1e-03 | loss_phys = 0.0e+00
It   6/21 | mse = 4.52e-08 | loss = 5.14e-03 | dt = 65.3s | lr = 4.7e-02 | loss_mse = 4.5e-03 | loss_phys = 0.0e+00
It   7/21 | mse = 4.30e-08 | loss = 4.47e-03 | dt = 52.5s | lr = 3.3e-02 |

sh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s python3 run_finn_core2.py /Users/r/Documents/stud/ss24/finnpinn/dsh: /dev/tty: Device not configured
sh: /dev/tty: Device not configured
94% 17:1=8m05s p

In [ ]:
!say "Done"

In [ ]:
# for quantile, bound_up, bound_down in p3inn_core2_dir.iter_pred_PIs():
#     for bound_type, bound in [("up", bound_up), ("down", bound_down)]:
#         finn_p3inn_core2_dir = FinnDir(finn_exp_root / f"{bound_type}_{quantile:g}")
#         np.save(finn_p3inn_core2_dir.c_train_path, np.squeeze(bound))
#         t = np.linspace(
#             data_core2_df["time"].min(), data_core2_df["time"].max(), len(bound)
#         )
#         np.save(finn_p3inn_core2_dir.t_train_path, t)

#         print(f"{finn_p3inn_core2_dir.path.relative_to(finn_exp_root)}")
#         finn_fit_retardation(
#             out_dir=finn_p3inn_core2_dir.path,
#             is_exp_data=True,
#             n_epochs=21,
#             **conf_core2,
#         )

# Analyze

TODO

### Compare initial 95% PI3NN PI with "FINN-Data-Noise PI"
### Compare with MCMC PI